# Data preprocessing

In [ ]:
import numpy as np
import math
import pandas as pd
import os
import matplotlib.pyplot as plt
import kagglehub

Read Dataset

In [ ]:
path = kagglehub.dataset_download("ashishpatel26/facial-expression-recognitionferchallenge")
df =  pd.read_csv(os.path.join(path,"fer2013", "fer2013", "fer2013.csv"), delimiter=",", header=0)
df.head()

Figure out the dimension of the images (48x48)

In [ ]:
len(df["pixels"][0])
math.sqrt(len(df.pixels[0].split(" ")))

Code to check wether the number of pixels of any of the images in the dataset aren't perfect squares

In [ ]:
pixel_count = len(df.pixels[0].split(' '))

for i in range(0, len(df["pixels"])):
    if not math.sqrt(pixel_count).is_integer():
        print(i)

expected_pixels = 48 * 48
for i in range(0, len(df["pixels"])):
    if pixel_count != expected_pixels:
        print(i)

In [ ]:
fig = plt.figure(1, (14, 14))

k = 0
for label in sorted(df["emotion"].unique()):
    for j in range(7):
        px = df[df["emotion"]==label]["pixels"].iloc[k]
        px = np.array(px.split(" ")).reshape(48, 48).astype("float32")

        k += 1
        ax = plt.subplot(7, 7, k)
        ax.imshow(px)
        ax.set_xticks([])
        ax.set_yticks([])
        plt.tight_layout()

Converts all strings of pixels into a numpy-array of floating point numbers

In [ ]:
df["pixels"] = df["pixels"].apply(lambda x: np.array(x.split(" ")).astype("float32"))
df.head()

Apply gaussian blur to all the images

In [ ]:
import cv2

df["pixels"] = df["pixels"].apply(lambda x: cv2.GaussianBlur(x, (3, 3), 0))
df.head()

In [ ]:
df["pixels"] = df["pixels"].apply(lambda x: x.reshape(48, 48))
df.head()

In [ ]:
plt.imshow(df["pixels"][0].astype(np.uint8))

Normalize pixel values to between 0 and 1

In [ ]:
df["pixels"] = df["pixels"].apply(lambda x: x / 255.0)
df["pixels"][0]

Split dataset into testing and training sets

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(df["pixels"], df["emotion"], test_size=0.2, random_state=1)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=2)
X_train.shape, X_test.shape, X_val.shape, y_train.shape, y_test.shape, y_val.shape


# Image classification using a CNN

Convert datasets into correct format

In [ ]:
# Convert to 3D array
X_train_CNN = np.stack(X_train)
X_test_CNN = np.stack(X_test)
X_val_CNN = np.stack(X_val)

# Add channel dimension
X_train_CNN = X_train_CNN[..., np.newaxis]
X_val_CNN = X_val_CNN[..., np.newaxis]
X_test_CNN = X_test_CNN[..., np.newaxis]

y_train_CNN = np.array(y_train)
y_test_CNN = np.array(y_test)
y_val_CNN = np.array(y_val)

Build and compile model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input, BatchNormalization
from tensorflow.keras.optimizers import Adam


model = Sequential()
model.add(Input(shape=(48, 48, 1)))

model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2, 2))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2, 2))
model.add(Dropout(0.25))

model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(2, 2))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

model.summary()


model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train_CNN, y_train_CNN, epochs=30, validation_data=(X_val_CNN, y_val_CNN), batch_size=32)

Evaluation

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# Plot Training History
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

# Evaluate on Test Data
test_loss, test_accuracy = model.evaluate(X_test_CNN, y_test_CNN)
print(f'Test Accuracy: {test_accuracy:.4f}')

y_pred = np.argmax(model.predict(X_test_CNN), axis=1)

print(classification_report(y_test_CNN, y_pred))

Confusion matrix

In [ ]:
cm = confusion_matrix(y_test_CNN, y_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=np.unique(df["emotion"].values), yticklabels=np.unique(df["emotion"].values))
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()